In [ ]:
import numpy as np
import time

%load_ext autoreload
%autoreload 2

In [ ]:
from multirtd.dubins_model import dubins_traj

dt = 0.1
T = 1.5  # 15 steps
N = int(T / dt)
init_pose = np.array([0, 0, 0])
goal = np.array([5, 0])

U = np.tile(np.array([1, 0]), (N, 1))
start_time = time.time()
for i in range(10000):
    dubins_traj(init_pose, U, dt)
print("Time elapsed: {:.3f} s".format(time.time() - start_time))

# TODO: add braking maneuver

In [ ]:
# Trajectory parameter space
v_max = 0.25  # [m/s]
w_max = 1.0  # [rad/s]

In [ ]:
# Obstacle
obs_c = np.array([1.2, 0])
obs_r = 1

### Sampled-based

In [ ]:
# Sample-based
from multirtd.utils import rand_in_bounds

start_time = time.time()
n_samples = 1000
u_samples = rand_in_bounds([-v_max, v_max, -w_max, w_max], n_samples)
endpoints = np.zeros((n_samples, 2))
for i, u in enumerate(u_samples):
    U = np.tile(u, (N, 1))
    traj = dubins_traj(init_pose, U, dt)
    endpoints[i] = traj[-1,:-1]

dists = np.linalg.norm(endpoints - goal, axis=1)
sort_idxs = np.argsort(dists)
u_samples_sorted = u_samples[sort_idxs]

# Check collisions
for u in u_samples_sorted:
    U = np.tile(u, (N, 1))
    traj = dubins_traj(init_pose, U, dt)
    d_vec = np.linalg.norm(traj[:,:-1] - obs_c, axis=1)
    if any(d_vec <= obs_r):
        continue
    else:
        print("found plan ", u)
        break
print("Time elapsed: {:.3f} s".format(time.time() - start_time))

In [ ]:
dubins_traj(init_pose, np.tile(u, (N, 1)), dt)

### Scipy optimize

In [ ]:
# Optimization-based
from scipy.optimize import minimize, NonlinearConstraint

goal = np.array([5, 2])

def cost(u):
    U = np.tile(u, (N, 1))
    traj = dubins_traj(init_pose, U, dt)
    dist = np.linalg.norm(traj[-1,:-1] - goal)
    return dist

def constraint(u):
    U = np.tile(u, (N, 1))
    traj = dubins_traj(init_pose, U, dt)
    dist = np.linalg.norm(traj[:,:-1] - obs_c, axis=1)
    return dist

start_time = time.time()
cons = NonlinearConstraint(constraint, obs_r, np.inf)
res = minimize(cost, np.array([0, 0]), method='SLSQP', constraints=cons, options={'disp': True})
print("Time elapsed: {:.3f} s".format(time.time() - start_time))
print(res.x)

In [ ]:
dubins_traj(init_pose, np.tile(res.x, (N, 1)), dt)

### With zonotope constraints

In [ ]:
import matplotlib.pyplot as plt

import multirtd.utils as utils
from multirtd.zonotope import Zonotope

obs_c = np.array([1.2, 0])[:,None]
obs_G = np.eye(2)
obs = Zonotope(obs_c, obs_G)

fig, ax = plt.subplots(figsize=(10,10))

obs.plot(ax, color='r')